# loading model 
this part needs to output:
1. tokenizer
2. model 
3. max_len

In [1]:
#tramsformers imports
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

from packaging import version
assert version.parse(transformers.__version__) >= version.parse("4.23.0")

In [2]:
#model parameters
DEVICE='cpu'
MAX_INPUT_LEN=4096

In [3]:
#imports
import json
import os
from os.path import join
import pandas as pd
from tqdm.notebook import tqdm

import pygments
from pygments.lexers import get_lexer_by_name

import torch.nn.functional as F 
import torch 


In [4]:
#genral parameters
data_file='cpp000000000302.json'
LANG='cpp'
SAVE_DIR='test_save'

In [5]:
#geting the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("NinedayWang/PolyCoder-160m")
model = AutoModelForCausalLM.from_pretrained("NinedayWang/PolyCoder-160m").to(DEVICE)

MAX_LEN=model.config.max_position_embeddings

In [6]:
#def make_internal_mask(model=model,l=MAX_INPUT_LEN):
    #making the attention mask with the corect context window
b = torch.ones(MAX_INPUT_LEN, MAX_INPUT_LEN,dtype=torch.bool)
b = torch.tril(b)
for i in range(MAX_INPUT_LEN):
    if(i>MAX_LEN):
        b[i][:i-MAX_LEN]=0
b = b.unsqueeze(0).unsqueeze(0).to(DEVICE)

for l in model.base_model.layers:
    l.attention.bias=b.clone()
        
#make_internal_mask()

In [7]:
import json

def process_data_file(data_file):
    """
    Read a file containing JSON objects, parse them, and handle any errors encountered.

    Args:
        data_file (str): The path to the data file.

    Returns:
        tuple: A tuple containing the processed data, a list of errors encountered,
               and a list of faulty lines.

    """
    data = []  # List to store the parsed JSON objects
    errors = []  # List to store the encountered errors
    faultys = []  # List to store the faulty lines

    with open(data_file, 'rb') as f:
        for i, line in enumerate(f):
            try:
                data.append(json.loads(line))  # Parse the JSON object and add it to 'data'
            except Exception as e:
                print(f'Errored at line {i}: {e}')
                errors.append(e)
                faultys.append(line)

    #print(f'data: {len(data)} errors: {len(errors)}')
    return data, errors, faultys

# Example usage:

repos, errors, faulty_lines = process_data_file(data_file)
print(f'data: {len(repos)} errors: {len(errors)}')

data: 10108 errors: 0


In [8]:
valid_repos=[d for d in repos if 'content' in d.keys() and 'repo_name' in d.keys()]
#codes=[d['content'] for d in valid_repos]
print(len(valid_repos))

10104


In [9]:
lexer = get_lexer_by_name(LANG)
LEX_VOCAB=sum(len(v) for v in lexer.tokens.values())
LEX_VOCAB

46

# chunked infrance

In [10]:
c=valid_repos[71]['content']
input_ids = tokenizer.encode(c, return_tensors='pt').to(DEVICE)
input_ids.shape,MAX_LEN,MAX_INPUT_LEN

(torch.Size([1, 33564]), 2048, 4096)

In [15]:
@torch.no_grad()
def run_model(input_ids):
    x=input_ids[:,:MAX_INPUT_LEN]
    #print(x.shape)
    out=model(x)
    ans=out.logits
    
    start=MAX_INPUT_LEN
    end=2*MAX_INPUT_LEN-MAX_LEN
    
    while start<input_ids.shape[1]:
        #print(out.past_key_values[1][0].shape)
        cut_kv=[[z[:,:,-MAX_LEN:] for z in y] for y in out.past_key_values]
        #print(cut_kv[0][0].shape)
        x=input_ids[:,start:end]
        #print(x.shape)
        out=model(x,past_key_values=cut_kv)
        ans=torch.cat([ans,out.logits],1)
        #print(ans.shape)
        start=end
        end+=MAX_INPUT_LEN-MAX_LEN
    
    return ans

ans=run_model(input_ids)
assert ans.shape[1]==input_ids.shape[1]
print('done unit test')

KeyboardInterrupt: 

# main loop

In [16]:
@torch.no_grad()
def compute_repo_metrics(repo):
    c = repo['content']
    num_tok = len(list(pygments.lex(c, lexer)))
    input_ids = tokenizer.encode(c, return_tensors='pt').to(DEVICE)
    out = run_model(input_ids)
    l = F.cross_entropy(out, F.one_hot(input_ids, out.shape[-1]).to(float), reduction='sum')
    return {'repo_name': repo['repo_name'], 'cross_entropy': l.detach().cpu().item(), 'lexical_length': num_tok}

compute_repo_metrics(valid_repos[0])

{'repo_name': 'VRToxin/android_external_stagefright-plugins',
 'cross_entropy': 140487.70995995402,
 'lexical_length': 3813}

In [ ]:
ans=[]
skiped=[]
for r in tqdm(valid_repos ):
    try: 
        ans.append(compute_repo_metrics(r))
    except Exception as e:
        print(f"error at {r['repo_name']}")
        skiped.append({'repo_name':r['repo_name'],'error':e})

  0%|          | 0/10104 [00:00<?, ?it/s]

In [ ]:
#raise skiped[-1]['error']
len(skiped)

In [ ]:
def get_results(ans):
    #need to do numerical stabilety better
    all_tokens=sum(x['lexical_length'] for x in ans)
    neg_log=torch.sum(torch.Tensor(tuple(x['cross_entropy'] for x in ans)))
    preplexity=torch.exp(neg_log/(all_tokens*LEX_VOCAB))
    return {'preplexity':preplexity.detach().cpu().item(),'lex tokens':all_tokens}

results=get_results(ans)
results

# record keeping

In [ ]:
good_df=pd.DataFrame(ans)
bad_df=pd.DataFrame(skiped)
ans_df=pd.DataFrame([results])

if SAVE_DIR:
    os.makedirs(SAVE_DIR)
    good_df.to_csv(join(SAVE_DIR,'calc.csv'),index=False)
    bad_df.to_csv(join(SAVE_DIR,'error.csv'),index=False)
    ans_df.to_csv(join(SAVE_DIR,'result.csv'),index=False)